In [1]:
from selenium import webdriver
from spacy.matcher import Matcher
import re
import spacy
import time
import pandas as pd 
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label
from ipywidgets import interact, interact_manual

In [ ]:
#pip install ipywidgets
#pip install selenium
#pip install spacy
#pip install pandas

In [2]:
def onlynumbers(gimmestring):
    num = ["0","1","2","3","4","5","6","7","8","9"]
    val = ""
    for letter in gimmestring:
        if(letter in num):
            val = val+letter
    return val

In [3]:
### Funções responsáveis por efetuar uma extração mais "inteligente" das informações brutas capturadas pelo robô 
### das páginas dos diários oficiais.

## Instala e carrega o idioma português no spacy
#!python -m spacy download pt_core_news_sm ## Descomentar na primeira vez que rodar (caso não tenha o pacote já instalado)
## Depois que instalar, resetar o kernel e comentar a linha de cima

nlp = spacy.load("pt_core_news_sm")


##### FUNÇÕES PARA EXTRAÇÃO DAS INFORMAÇÕES DO CAMPO "TITLE" #####


## Extração dos campos nomeDiário, data e numeracaoPagina do campo "Title"
def extract_data_from_title(strTitle):
    
    ## Transforma o tipo str no tipo doc
    docTitle = nlp(u''+strTitle)
    
    ## Campo nomeDiario
    diaryName = strTitle.split('-')[-1]
    
    ## Campo data
    date = extract_date(docTitle)
    
    ## Campo numeracaoPagina
    page = extract_page_number(docTitle)
    
    return diaryName, date, page
    

## Extração da data no formato string
def extract_date(nlp_doc):
    matcher = Matcher(nlp.vocab)

    ## Padrão: XX/XX/XXXX
    datePattern = [{"SHAPE": "dd/dd/dddd"},]

    ## Procura pelo padrão na entrada nlp_doc
    matcher.add('DATE', None, datePattern)
    matches = matcher(nlp_doc)

    ## Verifica se foi encontrado o padrão de data e retorna a data
    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        return span.text
    else:
        ## No caso de não encontrar data, retorna string vazia
        return ''

## Extração do numero da página no formato string
def extract_page_number(nlp_doc):
    matcher = Matcher(nlp.vocab)

    ## Padrões: Pág. X, Pág. XX, Pág. XXX, Pág. XXXX
    pageNumberPattern1 = [{"LOWER": "pág"}, {"IS_PUNCT": True}, {"SHAPE": "d"}]
    pageNumberPattern2 = [{"LOWER": "pág"}, {"IS_PUNCT": True}, {"SHAPE": "dd"}]
    pageNumberPattern3 = [{"LOWER": "pág"}, {"IS_PUNCT": True}, {"SHAPE": "ddd"}]
    pageNumberPattern4 = [{"LOWER": "pág"}, {"IS_PUNCT": True}, {"SHAPE": "dddd"}]

    ## Procura pelos padrões na entrada nlp_doc
    matcher.add('PAGE_NUMBER', None, pageNumberPattern1, pageNumberPattern2, 
                pageNumberPattern3, pageNumberPattern4)
    matches = matcher(nlp_doc)

    ## Verifica se foi encontrado o padrão de página e extrai apenas o número da página
    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        foundPattern = span.text
        return foundPattern.split(' ')[1]
    else:
        ## No caso de não encontrar o padrão, retorna vazio (sem página)
        return ''


    
##### FUNÇÕES PARA EXTRAÇÃO DAS INFORMAÇÕES DO CAMPO "BODY" #####

    
## Extração do decreto no formato string
def extract_decree(nlp_doc):
    matcher = Matcher(nlp.vocab)
    decreePatern1 = [{"LOWER": "art"}]

    matcher.add('DECREE_NUMBER', None, decreePatern1)

    matches = matcher(nlp_doc)
    occurrenceL = []

    for match_id, start, end in matches:
        span = nlp_doc[start:end]
        occurrenceL.append(span.text)
    return occurrenceL


## Verifica se há citação de algum decreto em cada linha de um documento.
## Se houver, então a linha é totalmente capturada.
## Entrada "conteudo" representa o campo "body"

def capture_decree_activities(content):

    activities = []

    for line in str(content).split('\nArt.'):
        
        ## Criação do documento
        document = nlp(u''+line)

        ## Verifica se algum decreto é citado na linha
        result = extract_decree(document)

        ## Caso houver citação de um decreto na linha
        if 'estabelecimento' in line:
            activities.append(line)

    return activities

In [4]:
def searchcount():
    count = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[1]/div[1]/div[1]")
    
    return count.text

In [5]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

path = "chromedriver.exe"

driver = webdriver.Chrome(executable_path=path, options=options)
time.sleep(3)

In [60]:
# Esta função cria uma lista com as informações dentro da página de busca do datajus

# datajus/diarios/

# title = titulo do artigo
# publisher = publicante do artigo
# body = conteúdo da página encontrada com a keyword referenciada
# link = link do artigo 

# diary = nome do diário extraído do título
# date = data de publicação do diário extraída do título
# page = página do diário extraído do título

def page_list():

    ## Lista com as informações brutas da página de busca
    dataL = []    
    
    #document_folder é a DIV dentro do HTML que representa todos os documentos dentro  da página
    #document_children serão os elementos filhos de 1º grau do elemento document_folder (cada um dos documentos)
    
    #usaremos isso para criar a condicional de visualização do crawler
    document_folder = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]")
    document_children = document_folder.find_elements_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/*")
    
    x = 1
    for i in document_children:
        #se o atributo "class" do elemento filho se chamar 'searchpaginator', signfica que ele é o page_selector
        if((i.get_attribute('class')) == 'Pagination pagination SearchPaginator'):
            break
        publisher = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/div[1]".format(x))
        
        title = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/h2".format(x))
        body = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/div[2]".format(x))
        link = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[2]/div[2]/div[{}]/div/h2/a".format(x))
        link_text = link.get_attribute("hre
        ## Extração dos campos NomeDiario, Data e Página
        diary, date, page = extract_data_from_title(title.text)
        
        dataL.append([diary, date, page, link_text, body.text])
        x = x+1
    
    return dataL

In [7]:
#pesquisa todos os filtros de jurisprudencia dentro do site
def jurisprudencia():
    jurisprudencia_mais = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[1]/div/div/div[3]/label")
    jurisprudencia_mais.click()
    
    jurisprudencia = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[1]/div/div/div[3]")
    juris_children = jurisprudencia.find_elements_by_class_name('DefaultFilter-item')
    
    texto = []
    value = []
    
    jurisprudencia = pd.DataFrame()
    
    texto.append('none')
    value.append('')
    
    for i in juris_children:
        texto.append(i.text)
        value.append(i.get_attribute('data-value'))
        
    jurisprudencia["descricao"] = (texto)
    jurisprudencia["value"] = (value)
    return jurisprudencia

In [8]:
def URL():
    article_type = driver.find_element_by_xpath("//*[@id='app-root']/div/div/div[1]/div[1]/div/div/div[4]")
    article_children = article_type.find_elements_by_class_name('DefaultFilter-item')
    
    texto = []
    href = []
    
    url_list = pd.DataFrame()
    
    for i in article_children:
        texto.append(i.text)
        href.append(i.get_attribute('data-artifact'))
    url_list["descricao"] = (texto)
    url_list["url"] = (href)
    
    url_list["url"][0] = 'none'
    
    return url_list

In [9]:
driver.get("https://www.jusbrasil.com.br/diarios/busca?q="+"none")

In [10]:
jurisprudencia = jurisprudencia()

In [11]:
url_list = URL()

In [135]:
def find_diarios(URL,keyword,other):
    df_all = pd.DataFrame()
    
    pages = 1
    result_num = 60
    while len(df_all) < result_num:
        search = "busca?q="+keyword+"&p="+str(pages)+other
        if(pages==1):
            driver.get(URL+"busca?q="+keyword+other)
            
            result_count = searchcount()
            result_num = int(onlynumbers(result_count))
            print(str(result_num)+" resultados encontrados")
            
            pages_txt = int(result_num/10)+1
            print(str(pages_txt)+" paginas para crawlear")
        else:
            driver.get(URL+search)
        infoL = page_list()
        try:
            df_all = df_all.append(infoL)
            print(str(len(df_all))+" resultados encontrados")
        except:
            print("pagina "+str(pages)+" sem resultados")
        
        pages += 1
        
    return df_all

In [138]:
@interact_manual
def choose(escolha_url=url_list["descricao"],
          date_from = widgets.DatePicker(
            description="from",
            disabled=False),
          date_to = widgets.DatePicker(
            description="to",
            disabled=False),
          keyword = widgets.Text(
            value="covid+fechamento",
            description="keywords"),
          juris = widgets.SelectMultiple(
            options=jurisprudencia["descricao"],
            value=['none'],
            disabled=False,
            style={'description_width': 'initial'})):
    
    for i in range(len(url_list)):
        if(escolha_url==url_list.iloc[i]["descricao"]):
            url = (url_list.iloc[i]["url"])
    
    if(date_from and date_to):
        other = ("&dateFrom="+str(date_from)+"&dateTo="+str(date_to))
    else:
        other = ''
    
    if(url=='none'):
        df_all = find_diarios('https://www.jusbrasil.com.br/diarios/','covid+fechamento',other)
        df_all = df_all.reset_index()
        df_all.to_excel(r'Result.xlsx', )
        return df_all
    else:
        string = 'https://www.jusbrasil.com.br/'+url
        df_all = find_diarios(string,'covid+fechamento',other)
        df_all = df_all.reset_index()
        df_all.to_excel(r'Result.xlsx', )
        return df_all

interactive(children=(Dropdown(description='escolha_url', options=('Tudo', 'Artigos', 'Notícias', 'Jurisprudên…